# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import numpy as np #Data manipulation
import pandas as pd #Point data manipulation and organization
import xarray as xr #Raster data manipulation and organization

import pathlib  #For filepaths, io, etc.
import os       #For several system-based commands
import datetime #For manipulation of time data, including file creation/modification times
import json     #For dictionary io, etc.

import matplotlib.pyplot as plt #For plotting and data vizualization
import geopandas as gpd         #For organization and manipulation of vector data in space (study area and some data points)
import rioxarray as rxr         #For orgnaization and manipulation of raster data
from scipy import interpolate
import shapely                  #For converting coordinates to point geometry

#Scripts with functions made for this specific application
import w4h

#Variables needed throughout, best to just assign now
todayDate, dateSuffix = w4h.getCurrentDate() 
repoDir = pathlib.Path(os.getcwd())

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [2]:
directoryDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\\'[:-1]
downholeDataPATH, headerDataPATH, xyzInPATH  = w4h.filesSetup(db_dir=directoryDir)

#Functions to read data into dataframes. Also excludes extraneous columns, and drops header data with no location information
headerDataIN, downholeDataIN = w4h.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH) 
xyzDataIN = w4h.readXYZData(xyzfile=xyzInPATH)

Most Recent version of this file is : ISGS_DOWNHOLE_DATA_2023-01-06.txt
Most Recent version of this file is : ISGS_HEADER_2023-01-06.txt
Most Recent version of this file is : xyzData.csv
Using the following files:

\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_DOWNHOLE_DATA_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_HEADER_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\xyzData.csv
Downhole Data has 3054409 valid well records.
Header Data has 636855 unique wells with valid location information.


Define datatypes (doing this during the read in process has presented issues)

In [3]:
#Define datatypes, to read into defineDataTypes() function
#Define datatypes of each column of the new dataframes
downholeDataIN = w4h.defineDataTypes(downholeDataIN, dtypeFile='downholeDataTypes.txt')
headerDataIN = w4h.defineDataTypes(headerDataIN, dtypeFile='headerDataTypes.txt')
xyzDataIN = w4h.defineDataTypes(xyzDataIN, dtypeFile='xyzDataTypes.txt')

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\read.py:145: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,i] = dfIN.iloc[:,i].astype(dtypes[dfIN.iloc[:,i].name])


Read in Study Area

In [4]:
import w4h
studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESL_StudyArea_5mi.shp"
#studyAreaPath = r"C:\Users\balikian\OneDrive - University of Illinois - Urbana\Data_OneDrive\CodesScripts\MahometBoreholesPolygon.zip"
#studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\WellData\ChicagoInset_Bndy_JFT.shp"
studyAreaIN = w4h.read_study_area(studyAreaPath)

Read in/Define Model Grid, surface elevation grid, and bedrock elevation grid

In [38]:
readGrids=True

if readGrids:
    modelGridPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\grid_625_raster.tif"
    surfaceElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ILStateLidar_ClipExtentESL.tif"
    bedrockElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESLBedrock.tif"

    modelGrid = w4h.read_grid(datapath=modelGridPath, grid_type='model', studyArea=studyAreaIN,  read_grid=True, clip2SA=True)#, gridcrs='EPSG:26715', studyAreacrs='EPSG:26715')
    surfaceElevGridIN = w4h.read_grid(datapath=surfaceElevPath, grid_type='surface', studyArea=studyAreaIN, use_service=False, clip2SA=True)
    bedrockElevGridIN = w4h.read_grid(datapath=bedrockElevPath, grid_type='bedrock', studyArea=studyAreaIN, use_service=False, clip2SA=True)

Add in Control points

In [6]:
#NEED CODE HERE FOR ADDING IN CONTROL Wells MANUALLY
#Add control headerInfo
#Add control description info

# Extract Elevation Data

Extract elevation data from consistent elevation dataset for all wells (lidar or other statewide DEM)

In [7]:
#First, get wells with updated xyz info
    #Check first if xyzData needs to be updated with locations (?)
    #Check which wells in headerData don't have associated lidar data

#statewideLidar =  ow
#mapping.rastertoPoints_extract()

Merge elevation data with headerData table

In [8]:
uniqueWells = headerData['API_NUMBER'].unique()
#xyzData['UniqueWells'] = uniqueWells

headerData = w4h.addElevtoHeader(xyzData, headerData)
##NEED TO UPDATE THIS TO WORK WITH DATA WITH NO XYZ ELEVATION DATA FROM LIDAR
#Change xyz column name to indicate lidar
#Use order of preference: lidar, headerData table?/30/10m DEM?

# Data Cleaning

## First, let's clean up records in the data without the necessary information

Clip data from outside Study Area

In [9]:
headerData = w4h.coords2Geometry(df=headerData, xCol='LONGITUDE', yCol='LATITUDE', crs='EPSG:4269')
#headerData['geometry']=headerData['GEOMETRY'].copy() #old code
headerDataClip = w4h.clipHeader2StudyArea(studyarea=studyAreaIN, headerdata=headerData, headerCRS='EPSG:4269')
headerData = headerDataClip.copy()
headerData

c:\Users\riley\anaconda3\envs\geospatial38\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry
0,13626212,201.0,15.0,2.0,N,5.0,W,3.0,NE NW NW,591.0,nan,27.0,nan,38.733337,-89.933357,580.419373,POINT (-89.93336 38.73334)
1,930917112,73.0,31.0,6.0,N,8.0,W,3.0,nan,537.0,GL,119.0,nan,38.930069,-90.030655,529.981140,POINT (-90.03065 38.93007)
2,930921712,75.0,25.0,6.0,N,9.0,W,3.0,SE,532.0,GL,119.0,nan,38.941998,-90.045364,530.172913,POINT (-90.04536 38.94200)
3,930921812,295.0,6.0,5.0,N,8.0,W,3.0,N2 SW NW,526.0,GL,119.0,nan,38.918377,-90.038200,520.954590,POINT (-90.03820 38.91838)
4,930921912,2195.0,23.0,5.0,N,8.0,W,3.0,SE,507.0,GL,119.0,nan,38.870552,-89.955078,507.976868,POINT (-89.95508 38.87055)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8145,1374073512,52.0,23.0,1.0,N,9.0,W,3.0,nan,NaN,nan,163.0,nan,38.520279,-90.058060,555.110840,POINT (-90.05806 38.52028)
8146,1374073812,70.0,33.0,2.0,N,9.0,W,3.0,SW NE SW,NaN,nan,163.0,nan,38.575832,-90.105003,409.501556,POINT (-90.10500 38.57583)
8147,1374073912,31.0,19.0,1.0,N,10.0,W,3.0,nan,413.0,DM,163.0,nan,38.528660,-90.244118,410.597595,POINT (-90.24412 38.52866)
8148,1374794512,61.0,12.0,2.0,N,9.0,W,3.0,NW,427.0,GL,163.0,nan,38.640568,-90.051552,431.323914,POINT (-90.05155 38.64057)


Now, remove data from downholeData table that does not have location information (Since we would not know where to put it anyway)

This should also essentially "clip" the downholeData to the study area, since only study area wells remain in headerData

In [10]:
downholeData = w4h.removeNonlocatedData(downholeData, headerData)

2998078 records removed without location information.
56331 wells remain from 7188 located wells in study area.


Remove headerData rows without surface elevation information (this currently clips data from outside Illinois)

In [11]:
headerData = headerDataClip.copy()
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry
0,13626212,201.0,15.0,2.0,N,5.0,W,3.0,NE NW NW,591.0,nan,27.0,nan,38.733337,-89.933357,580.419373,POINT (-89.93336 38.73334)
1,930917112,73.0,31.0,6.0,N,8.0,W,3.0,nan,537.0,GL,119.0,nan,38.930069,-90.030655,529.981140,POINT (-90.03065 38.93007)
2,930921712,75.0,25.0,6.0,N,9.0,W,3.0,SE,532.0,GL,119.0,nan,38.941998,-90.045364,530.172913,POINT (-90.04536 38.94200)
3,930921812,295.0,6.0,5.0,N,8.0,W,3.0,N2 SW NW,526.0,GL,119.0,nan,38.918377,-90.038200,520.954590,POINT (-90.03820 38.91838)
4,930921912,2195.0,23.0,5.0,N,8.0,W,3.0,SE,507.0,GL,119.0,nan,38.870552,-89.955078,507.976868,POINT (-89.95508 38.87055)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8145,1374073512,52.0,23.0,1.0,N,9.0,W,3.0,nan,NaN,nan,163.0,nan,38.520279,-90.058060,555.110840,POINT (-90.05806 38.52028)
8146,1374073812,70.0,33.0,2.0,N,9.0,W,3.0,SW NE SW,NaN,nan,163.0,nan,38.575832,-90.105003,409.501556,POINT (-90.10500 38.57583)
8147,1374073912,31.0,19.0,1.0,N,10.0,W,3.0,nan,413.0,DM,163.0,nan,38.528660,-90.244118,410.597595,POINT (-90.24412 38.52866)
8148,1374794512,61.0,12.0,2.0,N,9.0,W,3.0,NW,427.0,GL,163.0,nan,38.640568,-90.051552,431.323914,POINT (-90.05155 38.64057)


In [12]:
headerData_cleaned = w4h.removenotopo(df=headerData, printouts=True)
headerData = headerData_cleaned.copy()

Well records removed: 0
Number of rows before dropping those without surface elevation information: 8150
Number of rows after dropping those without surface elevation information: 8150


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [13]:
#Drop records with no depth information
donwholeData = w4h.dropnodepth(downholeData, printouts=True)
#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
donwholeData = w4h.dropbaddepth(downholeData, printouts=True)

Number of rows before dropping those without record depth information: 56331
Number of rows after dropping those without record depth information: 55747
Number of well records without formation information deleted: 584
Number of rows before dropping those with obviously bad depth information: 56331
Number of rows after dropping those with obviously bad depth information: 55725
Well records deleted: 606


Drop records with no FORMATION information

In [14]:
downholeData = w4h.dropnoformation(downholeData, printouts=True)

Number of rows before dropping those without FORMATION information: 56331
Number of rows after dropping those without FORMATION information: 56331
Well records deleted: 0


Now we are going to export this data, to have record of cleaned data

In [15]:
downholeData = downholeData[downholeData['TABLE_NAME'] == 'HWYBRIDGE_LOG']
downholeData

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM
5773,931135312,HWYBRIDGE_LOG,hard gray sandy clayey silt,5.0,34.0,39.0
5774,931135312,HWYBRIDGE_LOG,hard mottled clayey silt,2.0,7.0,9.0
5775,931135312,HWYBRIDGE_LOG,stiff black clayey silt,2.5,0.0,2.5
5776,931135312,HWYBRIDGE_LOG,stiff dark gray silt,5.0,11.5,16.5
5777,931135312,HWYBRIDGE_LOG,stiff gray clayey silt,3.0,21.0,24.0
...,...,...,...,...,...,...
56326,1374794512,HWYBRIDGE_LOG,brown silt (levee fill),8.0,0.0,8.0
56327,1374794512,HWYBRIDGE_LOG,gray clay,5.0,11.5,16.5
56328,1374794512,HWYBRIDGE_LOG,gray clay (with thin sand streaks),7.5,26.5,34.0
56329,1374794512,HWYBRIDGE_LOG,"gray, fine sand",5.0,39.0,44.0


In [16]:
downholeData.reset_index(inplace=True,drop=True)
headerData.reset_index(inplace=True,drop=True)

#downholeData.to_csv(str(repoDir)+'/out/downholeData_cleaned'+dateSuffix+'.csv',index_label='ID')
#headerData.to_csv(str(repoDir)+'/out/headerData_cleaned'+dateSuffix+'.csv',index_label='ID')

In [17]:
outData = pd.merge(left = downholeData, right = headerData, on='API_NUMBER')
#downholeData = outData.copy()
downholeDataCLEANED = downholeData.copy()

# Classification

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: [Not defined]
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - no context

In [18]:
#Read in dictionary files for downhole data
specTermsPATH, startTermsPATH = w4h.searchTermFilePaths(dictdir=str(repoDir)+'/resources/', specStartPattern='*SearchTerms-Specific*', startGlobPattern = '*SearchTerms-Start*')

Most Recent version of this file is : SearchTerms-Specific_2022-11-16_essCols.csv
Most Recent version of this file is : SearchTerms-Start.csv


In [19]:
specTerms = w4h.read_dictionary_terms(dict_file=specTermsPATH)
startTerms = w4h.read_dictionary_terms(dict_file=startTermsPATH)
oldDictPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\WellData\Dictionaries\DICTIONARY_Updated-06-2018.csv"
oldDict = w4h.read_dictionary_terms(dict_file=oldDictPath, cols={'DESCRIPTION':'FORMATION', 'LITHOLOGY':'INTERPRETATION'}, class_flag=1)
specTerms = pd.concat([specTerms, oldDict])
specTerms.drop_duplicates(subset='FORMATION', inplace=True)
specTerms.reset_index(inplace=True, drop=True)
specTerms

c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\read.py:210: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  d.iloc[:,i] = d.iloc[:,i].astype(dict_termDtypes[d.iloc[:,i].name])
c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\read.py:210: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  d.iloc[:,i] = d.iloc[:,i].astype(dict_termDtypes[d.iloc[:,i].name])
c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\read.py:210: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setti

,FORMATION,INTERPRETATION,CLASS_FLAG
0,"sand, fine gr",SAND,1
1,"sand, fine-large gr",SAND,1
2,"sand, some large gr",SAND,1
3,toop soil,SOIL,1
4,grav in muck,GRAVEL,1
...,...,...,...
184290,CRS SAND GRAV AND ROCKS,SAND AND GRAVEL,1
184291,CRS SAND GRAV BOULDERS,SAND AND GRAVEL,1
184292,CRS SAND GRAV CLEAN,SAND AND GRAVEL,1
184293,carbonaceous shale,BEDROCK,1


Join the dataframes--for the specific search terms, this is the same as classifying them

In [20]:
downholeData= downholeDataCLEANED.copy()
downholeData_spec = w4h.specificDefine(downholeData, specTerms, printouts=True)
downholeData = downholeData_spec.copy()

0               hard gray sandy clayey silt
1                  hard mottled clayey silt
2                   stiff black clayey silt
3                      stiff dark gray silt
4                    stiff gray clayey silt
                        ...                
17945               brown silt (levee fill)
17946                             gray clay
17947    gray clay (with thin sand streaks)
17948                       gray, fine sand
17949         gray, mottled with brown clay
Name: FORMATION, Length: 17950, dtype: object
0               hard gray sandy clayey silt
1                  hard mottled clayey silt
2                   stiff black clayey silt
3                      stiff dark gray silt
4                    stiff gray clayey silt
                        ...                
17945               brown silt (levee fill)
17946                             gray clay
17947    gray clay (with thin sand streaks)
17948                       gray, fine sand
17949         gray, mottled wi

Create a dataframe with only the records already classified (using the specific search terms in this case, classifiedDF), and one that still needs to be searched (searchDF)

In [21]:
classifedDF, searchDF = w4h.splitDefined(downholeData)
searchDF.shape[0]

11106

Now, do the classification routine on the searchDF database

In [22]:
searchDF = w4h.startDefine(df=searchDF, starterms=startTerms, printouts=True)

Start Term process should be done by 14:23


c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\classify.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].where(~df['FORMATION'].str.startswith(s,na=False),4,inplace=True)
c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\classify.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INTERPRETATION'].where(~df['FORMATION'].str.startswith(s,na=False),starterms.loc[i,'INTERPRETATION'],inplace=True)


Records classified with start search term: 558
Records classified with start search term: 5.02% of remaining data


Merge specDF and searchDF back together all back in single dataframe

In [23]:
downholeData_Terms = w4h.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Terms.copy()

In [24]:
var = downholeData.CLASS_FLAG.value_counts()
downholeData.CLASS_FLAG.value_counts()

1.0    6844
4.0     558
Name: CLASS_FLAG, dtype: int64

Export terms that still need to be defined to csv (along with their counts)

In [25]:
#The outdir should be changed so it doesn't clog up the repository
#classify.export_toBeDefined(df=downholeData, outdir=str(repoDir)+'/out/')

Classify all  data under depth threshold (default is 550') as bedrock (should not be an issue, but just in case)

In [26]:
classifedDF, searchDF = w4h.splitDefined(downholeData)
searchDF = w4h.depthDefine(searchDF, thresh=550, printouts=True)
downholeData_Class = w4h.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Class.copy()

Records classified as bedrock that were deeper than 550': 0
This represents 0.0% of the unclassified data in this dataframe.


Add '0' flag for data still not classified

In [27]:
downholeData = w4h.fillUnclassified(downholeData)

In [28]:
downholeData['CLASS_FLAG'].value_counts()

0.0    10548
1.0     6844
4.0      558
Name: CLASS_FLAG, dtype: int64

In [ ]:
downholeData.to_csv(r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\WellData\out.csv")

## Add "Flag" for target interpratations

In [29]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
targetInterpDF = w4h.readLithologies()

In [30]:
downholeData = w4h.mergeLithologies(downholedata=downholeData, targinterps=targetInterpDF)

Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [31]:
downholeData['TARGET'].value_counts()

0.0    5171
1.0    2220
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [32]:
#Get Unique well APIs
wellsDF = w4h.getUniqueWells(downholeData)

Number of unique wells in downholeData: 2006


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [33]:
#Make this into a function?
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True, drop=True)
downholeData_sorted = downholeData_sorted[pd.notna(downholeData_sorted["INTERPRETATION"])]
donwholeData.reset_index(inplace=True, drop=True)
donwholeData = downholeData_sorted.copy()

# Get Bedrock Depth and Layer Thickness

Plot just to see them

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [39]:
inGrids = [bedrockElevGridIN, surfaceElevGridIN]
bedrockGrid, surfaceGrid = w4h.alignRasters(unalignedGrids=inGrids, modelgrid=modelGrid)

fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockGrid.plot(ax=ax[0])
surfaceGrid.plot(ax=ax[1])

Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [40]:
driftThickGrid, layerThickGrid = w4h.get_drift_thick(surface=surfaceGrid, bedrock=bedrockGrid, noLayers=9, plotData=False)

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [41]:
headerData = w4h.sample_raster_points(raster=bedrockGrid, ptDF=headerData, newColName='BEDROCK_ELEV_FT')
#headerData['BEDROCK_ELEV_M'] = headerData['BEDROCK_ELEV_FT']* 0.3048

headerData = w4h.sample_raster_points(raster=surfaceGrid, ptDF=headerData, newColName='SURFACE_ELEV_FT')
#headerData['SURFACE_ELEV_M'] = headerData['SURFACE_ELEV_FT']* 0.3048

headerData = w4h.sample_raster_points(raster=driftThickGrid, ptDF=headerData, newColName='BEDROCK_DEPTH_FT')
#headerData['BEDROCK_DEPTH_M'] = headerData['BEDROCK_DEPTH_FT']* 0.3048

headerData = w4h.sample_raster_points(raster=layerThickGrid, ptDF=headerData, newColName='LAYER_THICK_FT')
#headerData['LAYER_THICK_M'] = headerData['LAYER_THICK_FT']* 0.3048

BEDROCK_ELEV_FT sampling should be done by 14:29
SURFACE_ELEV_FT sampling should be done by 14:29
BEDROCK_DEPTH_FT sampling should be done by 14:29
LAYER_THICK_FT sampling should be done by 14:29


Calculate  all layer depths/elevations at all wells

In [42]:
headerData = w4h.get_layer_depths(well_metadata=headerData, no_layers=9)

Merge Data from downhole and headerData to enable further calculation

In [43]:
downholeData = downholeData_sorted.copy()
downholeData_layerInfo = w4h.merge_tables(data_df=downholeData, header_df=headerData,on='API_NUMBER', how='inner', auto_pick_cols=True)

Calculate the top and bottom elevation for each well record in downholeData

In [44]:
downholeData = downholeData_layerInfo.copy()

Define function to calculate target thickness in each layer

In [45]:
donwholeData['TARGET'].value_counts()

0.0    5171
1.0    2220
Name: TARGET, dtype: int64

In [54]:
resdf = w4h.layer_target_thick(downholeData_layerInfo, layers=9, outfile_prefix='CoarseFine')

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

Export to 

In [ ]:
layers_data = w4h.layer_interp(points=resdf, layers=9, grid=modelGrid, method='lin')

# Export

In [ ]:
def export_layers(layer_data, filepath, format):

    return

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")